In [ ]:
input_sizes = 
hidden_size = 

In [ ]:
with tf.variable_scope("linear"):
    initializer = tf.random_uniform_initializer(-1/math.sqrt(hidden_size), 1/math.sqrt(hidden_size))


    matrices = [tf.get_variable("matrix_%d" % (i,), (ipt_sz, self.hidden_size), initializer=initializer)
                     for i, ipt_sz in enumerate(self.input_sizes)]
    bias = tf.get_variable("bias", (self.hidden_size,), initializer=tf.constant_initializer(bias_init))

    inputs = flatten(inputs)

        summands = [tf.matmul(ipt,weight) for ipt, weight in zip(inputs, self.matrices)]
        summands.append(self.bias)
        return sum(summands)

In [ ]:
with tf.variable_scope("GRU"):
    self.reset_gate = Linear(self.input_sizes + [self.hidden_size], self.hidden_size, bias_init=1.0, scope="reset_gate")
    self.update_gate = Linear(self.input_sizes + [self.hidden_size], self.hidden_size, bias_init=1.0, scope="update_gate")
    self.memory_interpolation = Linear(self.input_sizes + [self.hidden_size], self.hidden_size, scope="memory_interpolation")

In [ ]:
    def __call__(self, inputs, state):
        inputs = flatten(inputs)
        with tf.variable_scope(self.scope):
            r = self.reset_gate(inputs + [state])
            u = self.update_gate(inputs + [state])
            r, u = tf.sigmoid(r), tf.sigmoid(u)
            c = self.memory_interpolation(inputs + [r * state])
            c = self.final_nonlinearity(c)
            return u * state + (1-u) * c

In [ ]:
    def zero_state(self, batch_size):
        zeros = tf.zeros(tf.pack([batch_size, self.hidden_size]))
        zeros.set_shape([None, self.hidden_size])
        return zeros

In [ ]:

r, u = array_ops.split(1, 2, linear([inputs, state], 2 * self._num_units, True, 1.0))
r, u = sigmoid(r), sigmoid(u)
c = tanh(linear([inputs, r * state], self._num_units, True))
new_h = u * state + (1 - u) * c
return new_h, new_h